## For Demo

In [ ]:
# cols=['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 
#          'green_0', 'green_2p', 'green_3p', 'green_4p', 'green_5p', 'green_6', 
#          'red_0', 'red_2p', 'red_3p', 'red_4p', 'red_5p', 'red_6', 
#          'blue_0', 'blue_2p', 'blue_3p', 'blue_4p', 'blue_5p', 'blue_6', 
#          'colours', 'total']
# df = pd.DataFrame(record_list, columns=cols)

# # Save the DataFrame as a CSV file
# df.to_csv('colour_records/rec_10.csv', index=False)

In [ ]:
import pandas as pd
import math
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time


# To send email notification --------------------------------------------------------------
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

notification_start = False
def email_notification():
    # Email details
    smtp_server = 'smtp.gmail.com'
    smtp_port = 587
    sender_email = 'chikwadoeze@gmail.com'
    receiver_email = 'cjnnabueze@gmail.com'
    app_password = 'zqwl eloj evel cvkp'
    subject = 'Time for Test-Trading'
    message = 'This is a Python test message from CJay'

    # Create a multipart message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    # Attach the message to the email
    msg.attach(MIMEText(message, 'plain'))

    # Create a secure connection with the SMTP server
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, app_password)
        server.send_message(msg)
        print('Email sent successfully!')
# -----------------------------------------------------------------------------------------



# Set up the Chrome driver service
service = Service("chrome_driver/chromedriver.exe")
driver = webdriver.Chrome(service=service)

# Create an instance of ActionChains
actions = ActionChains(driver)

# Open the specified URL
driver.get("https://logigames.bet9ja.com/Games/Launcher?gameId=11000&provider=0&pff=1&skin=201")
# driver.maximize_window()
time.sleep(5)

# Initialize an empty list to store records
record_list = []
start_trade = False
end_trade = False
martingale_start_level = 0
capital = 100_000
print(f"Capital = {capital}")
win_count = 0
max_loss = []

red_start, red_loss_count = False, 0
green_start, green_loss_count = False, 0
blue_start, blue_loss_count = False, 0

# MARTINGALE LIST GENERATOR -------------------------------------
target_percentage = 0.2
target = round(capital * (target_percentage / 100))
print(f"Profit Target = {target}")
odd = 3.8
martingale_levels = 22
martingale_loss = 0
martingale_stakes = []
for x in range(martingale_levels):
    stake = math.ceil((target + martingale_loss) / (odd - 1))
    martingale_stakes.append(stake)
    martingale_loss += stake
print(martingale_stakes)
# -------------------------------------------------------

def colour_stake():
    # print("START TRADE HERE ...")
    driver.find_element(By.XPATH, "//div[@class='stats__toggle active']").click()
    time.sleep(1)
   
    driver.find_element(By.XPATH, "//a[5]").click()  # Total Colour
    time.sleep(1)

    def stake_action(loss_count, colour_string):
        if loss_count < martingale_levels:
            stake_amount = martingale_stakes[loss_count]
            driver.find_element(By.XPATH, f"//div[@class='g-total__btn {colour_string} ']").click()  # Click on selected colour
            time.sleep(0.5)

            stake_box = driver.find_element(By.XPATH, "//body[1]/div[1]/div[1]/div[1]/div[1]/main[1]/div[2]/div[2]/div[3]/div[1]/div[2]/div[1]/div[1]/input[1]")
            stake_box.click()  # Click on stake box
            stake_box.send_keys(Keys.CONTROL + "a")  # Highlight the existing value in the stake box
            stake_box.send_keys(Keys.BACKSPACE)  # Clear the highlighted value
            time.sleep(0.5)
            
            stake_box.send_keys(stake_amount)  # Enter stake amount
            time.sleep(1)
            try:
                driver.find_element(By.XPATH, "//a[@class='place-bet']").click()  # Place bet
            except ElementClickInterceptedException:
                print("Place bet button delay ---------------------------------->>>")
                driver.refresh()  # Reload the current webpage
                time.sleep(1)
                driver.find_element(By.XPATH, "//a[5]").click()  # Total Colour
                driver.find_element(By.XPATH, f"//div[@class='g-total__btn {colour_string} ']").click()
                time.sleep(0.5)

                stake_box = driver.find_element(By.XPATH, "//body[1]/div[1]/div[1]/div[1]/div[1]/main[1]/div[2]/div[2]/div[3]/div[1]/div[2]/div[1]/div[1]/input[1]")
                stake_box.click()  # Click on stake box
                stake_box.send_keys(Keys.CONTROL + "a")  # Highlight the existing value in the stake box
                stake_box.send_keys(Keys.BACKSPACE)  # Clear the highlighted value
                # time.sleep(0.5)
                
                stake_box.send_keys(stake_amount)  # Enter stake amount
                time.sleep(0.5)
                driver.find_element(By.XPATH, "//a[@class='place-bet']").click()  # Place bet
            time.sleep(3)
            driver.find_element(By.XPATH, f"//div[@class='g-total__btn {colour_string} active']").click()  # to unselect the last picked colour
        else:
            print(f"Martingale Max Level reached -----> {loss_count} -- {colour_string}")

    val_2 = (green_start, red_start, blue_start)
    val_3 = (green_loss_count, red_loss_count, blue_loss_count)
    val_4 = ("green", "red", "blue")

    for start_colour, colour_loss_count, color in zip(val_2, val_3, val_4):
        if start_colour:
            stake_action(colour_loss_count, color)
    time.sleep(3)

    # print(driver.find_element(By.XPATH, "//div[@class='timeline__value-txt']").text)  # print counter
    # print("END TRADE HERE ...\n")

while True:
    # Get the counter value from the webpage
    counter = driver.find_element(By.XPATH, "//div[@class='timeline__value-txt']").text

    if int(counter) < 44 and int(counter) > 39:
        # Click on the statistics button
        driver.find_element(By.XPATH, "//div[@class='stats__toggle ']").click()     # -error check 1
        time.sleep(0.5)
        # Click on the "Rainbow" option in the dropdown menu
        driver.find_element(By.XPATH, "//a[contains(text(),'Total Colour')]").click()
        # Click on the "100 Draws" option in the dropdown menu
        driver.find_element(By.XPATH, "//div[@class='dd__trigger active dd__menu-open']//div[@class='dd__menu-item--value'][normalize-space()='100 Draws']").click()
        time.sleep(5)

        # Get the statistics text from the webpage
        stats = driver.find_element(By.XPATH, "//div[@class='p-stats__content']").text

        total_stats = tuple([int(x[-2:]) for x in stats.split("%")[:-1]])
        n = total_stats[0]

        # For sending email notification to start trading
        if (notification_start == False) and (n in [30, 31, 32]) and (n == max(total_stats)) and (total_stats.count(n) == 1) \
            and (min(total_stats) >= 22) and (max(total_stats[1:]) - min(total_stats[1:]) <= 4):
            email_notification() 
            notification_start = True

        # This gets the Last Drawn values on the Statistics page
        last_drawn_record = tuple([int(x.split(" ")[0].split("\n")[1]) for x in stats.split("%")[1:]])

        grb_last_drawn = last_drawn_record[1:]  # grb - green red blue last_drawn values
        green, red, blue = grb_last_drawn[0], grb_last_drawn[1], grb_last_drawn[2]
        if green_start == True:
            if green != 0:
                green_loss_count += 1
                if green_loss_count >= martingale_levels:
                    max_loss.append(green_loss_count)
                    print(f"green stop --> {green_loss_count}")
                    end_trade = True
                    green_start = False
                    
            else:
                max_loss.append(green_loss_count)
                green_loss_count = 0
                green_start = False
                win_count += 1 
                if end_trade and (any([red_start, blue_start]) == False):
                    print("END OF TRADE 1")
                    break            

        if red_start == True:
            if red != 0:
                red_loss_count += 1
                if red_loss_count >= martingale_levels:
                    max_loss.append(red_loss_count)
                    print(f"red stop --> {red_loss_count}")
                    end_trade = True
                    red_start = False
            else:
                max_loss.append(red_loss_count)
                red_loss_count = 0
                red_start = False
                win_count += 1
                if end_trade and (any([green_start, blue_start]) == False):
                    print("END OF TRADE 2")
                    break

        if blue_start == True:
            if blue != 0:
                blue_loss_count += 1
                if blue_loss_count >= martingale_levels:
                    max_loss.append(blue_loss_count)
                    print(f"blue stop --> {blue_loss_count}")
                    end_trade = True
                    blue_start = False
            else:
                max_loss.append(blue_loss_count)
                blue_loss_count = 0
                blue_start = False
                win_count += 1
                if end_trade and (any([green_start, red_start]) == False):
                    print("END OF TRADE 3")
                    break

        if green == martingale_start_level and end_trade == False:
            green_start = True
        if red == martingale_start_level and end_trade == False:
            red_start = True
        if blue == martingale_start_level and end_trade == False:
            blue_start = True

        if end_trade and (any([green_start, red_start, blue_start]) == False):
            print("END OF TRADE 4")
            break


        if 0 in grb_last_drawn and len(max_loss) > 0:
            print(f"Total Wins = {win_count} --> Last Loss count = {max_loss[-1]} --> Max Loss {max(max_loss)}")
   

        # Click on the "History" tab
        driver.find_element(By.XPATH, "//a[normalize-space()='History']").click()
        time.sleep(2)
        
        # Get the history text from the webpage
        history = driver.find_element(By.XPATH, "//div[@class='p-stats__content']").text
        
        # This gets the number of times each colour appears from RED, GREEN and BLUE respectively
        # E.g '213' means 2 reds, 1 green and 3 blues
        colours_count = history.split(" ")[9:][0].split("\n")[-1]

        # This gets the Total number count for all six balls
        # E.g 148 means total sum for the given balls - 22, 14, 4, 35, 26, 47 
        sum_colours = int(history.split(" ")[9:][1].split("\n")[0])

        # Get the numbers for all 6 balls
        ball_path = "/html[1]/body[1]/div[1]/div[1]/div[1]/div[1]/main[1]/div[4]/div[2]/table[1]/tbody[1]/tr[1]/td[2]/div[1]/span["
        ball_1 = int(driver.find_element(By.XPATH, f"{ball_path}1]").text)
        ball_2 = int(driver.find_element(By.XPATH, f"{ball_path}2]").text)
        ball_3 = int(driver.find_element(By.XPATH, f"{ball_path}3]").text)
        ball_4 = int(driver.find_element(By.XPATH, f"{ball_path}4]").text)
        ball_5 = int(driver.find_element(By.XPATH, f"{ball_path}5]").text)
        ball_6 = int(driver.find_element(By.XPATH, f"{ball_path}6]").text)
        
        # Record all collected results in a listed tuple
        all_balls = (ball_1, ball_2, ball_3, ball_4, ball_5, ball_6)
        all_record = all_balls + total_stats + last_drawn_record + (colours_count, sum_colours)
        record_list.append(all_record)
        # print(all_record)

        # START TRADING HERE
        colour_stake()  # Call the colour_stake function to start trading
        # start_trade = True

    else:
        time.sleep(1)  # 1 seconds counter delay

# Close the browser session
driver.quit()

In [ ]:
sorted([23, 32, 16, 25])

In [ ]:
# cols=['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 
#          'green_0', 'green_2p', 'green_3p', 'green_4p', 'green_5p', 'green_6', 
#          'red_0', 'red_2p', 'red_3p', 'red_4p', 'red_5p', 'red_6', 
#          'blue_0', 'blue_2p', 'blue_3p', 'blue_4p', 'blue_5p', 'blue_6', 
#          'colours', 'total']

cols=['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 
      'none_T100', 'green_T100', 'red_T100', 'blue_T100', 
      'none_LD', 'green_LD', 'red_LD', 'blue_LD', 'colours', 'total']
df = pd.DataFrame(record_list, columns=cols)
df['colours'] = df['colours'].astype(str)

# Save the DataFrame as a CSV file
df.to_csv('colour_records/rec_7_new.csv', index=False)

In [ ]:
print(record_list)

In [ ]:
df[df.duplicated()]

## For Real Trading

In [12]:
import pandas as pd
import pywhatkit
import math
import datetime
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time


# # To send email notification --------------------------------------------------------------
# import smtplib
# from email.mime.text import MIMEText
# from email.mime.multipart import MIMEMultipart

# notification_start = False
# def email_notification(message="Auto Trading Update Alert"):
#     # Email details
#     smtp_server = 'smtp.gmail.com'
#     smtp_port = 587
#     sender_email = 'chikwadoeze@gmail.com'
#     receiver_email = 'cjnnabueze@gmail.com'
#     app_password = 'zqwl eloj evel cvkp'
#     subject = 'TRADING UPDATE --> '

#     # Create a multipart message
#     msg = MIMEMultipart()
#     msg['From'] = sender_email
#     msg['To'] = receiver_email
#     msg['Subject'] = subject

#     # Attach the message to the email
#     msg.attach(MIMEText(message, 'plain'))

#     # Create a secure connection with the SMTP server
#     with smtplib.SMTP(smtp_server, smtp_port) as server:
#         server.starttls()
#         server.login(sender_email, app_password)
#         server.send_message(msg)
#         print('Email sent successfully!')
# # -----------------------------------------------------------------------------------------


# Set Chrome options to run the browser in the background
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode

service = Service("chrome_driver/chromedriver.exe")
driver = webdriver.Chrome(service=service, options=chrome_options)

# Create an instance of ActionChains
actions = ActionChains(driver)

# Open the specified URL
driver.get("https://logigames.bet9ja.com/Games/Launcher?gameId=11000&provider=0&pff=1&skin=201")
# driver.maximize_window()
time.sleep(5)

# Initialize an empty list to store records
record_list = []
start_trade = False
pause_trade = False
end_trade = False
martingale_start_level = 0
capital = 100_000
print(f"Capital = {capital}")
win_count_target = 100  # This is the number of times we want to Win
win_count = 0
max_loss = []
loss_count = 0
max_num = 11
min_num = 6
green_count, red_count, blue_count = [], [], []
colour_list = [green_count, red_count, blue_count]
draw_counter = 0
loss_count_list = []

none_high_check = 0

# MARTINGALE LIST GENERATOR -------------------------------------
target_percentage = 0.2
target = round(capital * (target_percentage / 100))
print(f"Profit Target = {target}")
odd = 3.8
martingale_levels = 22
martingale_loss = 0
martingale_stakes = []
for x in range(martingale_levels):
    stake = math.ceil((target + martingale_loss) / (odd - 1))
    martingale_stakes.append(stake)
    martingale_loss += stake
print(f"Martingale Levels - {martingale_stakes}")
print(f"Martingale Sum = {sum(martingale_stakes)}\n{'-'*50}")
Initial_balance = int(driver.find_element(By.XPATH, "//div[@class='rs-menu__balance-value']").text)
print(f"Initial Balance = {Initial_balance}")
# -------------------------------------------------------

# phone_number = "+2347048275297"  # Replace with the recipient's number with country code
# message = "This is a message sent using PyWhatKit."
# pywhatkit.sendwhatmsg_instantly(phone_number, message, tab_close=True, close_time=3) # Send the message immediately

while True:
    # Get the counter value from the webpage
    counter = driver.find_element(By.XPATH, "//div[@class='timeline__value-txt']").text

    if int(counter) < 41 and int(counter) > 37:
        draw_counter += 1
        # Click on the statistics button
        driver.find_element(By.XPATH, "//div[@class='stats__toggle ']").click()     # -error check 1
        time.sleep(0.5)
        
        driver.find_element(By.XPATH, "//a[contains(text(),'Total Colour')]").click()  # Click on the "Total Colour" option in the dropdown menu
        # Click on the "100 Draws" option in the dropdown menu
        driver.find_element(By.XPATH, "//div[@class='dd__trigger active dd__menu-open']//div[@class='dd__menu-item--value'][normalize-space()='100 Draws']").click()
        time.sleep(3.5)

        # Get the statistics text from the webpage
        stats = driver.find_element(By.XPATH, "//div[@class='p-stats__content']").text

        total_stats = tuple([int(x[-2:]) for x in stats.split("%")[:-1]])
        green_count.append(total_stats[1]), red_count.append(total_stats[2]), blue_count.append(total_stats[3])

        # This gets the Last Drawn values on the Statistics page
        last_drawn_record = tuple([int(x.split(" ")[0].split("\n")[1]) for x in stats.split("%")[1:]])
        if draw_counter > 1 and previous_stats_record == last_drawn_record:
            print("............... Duplication Last-Drawn correction ...............")
            green_count.pop(), red_count.pop(), blue_count.pop()
            driver.find_element(By.XPATH, "//a[normalize-space()='History']").click()  # Click on the "History" tab
            time.sleep(1.5)
            driver.find_element(By.XPATH, "//a[contains(text(),'Total Colour')]").click()  # Click on the "Total Colour" option in the dropdown menu
            time.sleep(0.5)
            driver.find_element(By.XPATH, "//div[@class='dd__trigger active dd__menu-open']//div[@class='dd__menu-item--value'][normalize-space()='100 Draws']").click()
            time.sleep(3)
            stats = driver.find_element(By.XPATH, "//div[@class='p-stats__content']").text
            total_stats = tuple([int(x[-2:]) for x in stats.split("%")[:-1]])
            green_count.append(total_stats[1]), red_count.append(total_stats[2]), blue_count.append(total_stats[3])
            last_drawn_record = tuple([int(x.split(" ")[0].split("\n")[1]) for x in stats.split("%")[1:]])
        previous_stats_record = last_drawn_record

        # grb_last_drawn = stats_record[1:]  # grb - green red blue last_drawn values
        # green_ld, red_ld, blue_ld = stats_record[1], stats_record[2], stats_record[3]

        # Check if result is a Win or Loss ----------------------------------------------------------------------------
        result_colours = [last_drawn_record[1], last_drawn_record[2], last_drawn_record[3]]
        if start_trade == True:
            if result_colours[colour_index] == 0:
                start_trade = False
                win_count += 1
                max_loss.append(loss_count)
                loss_count = 0
                # print(f"Total Wins = {win_count} --> Selected Colour = {colour_string} --> Last Loss count = {max_loss[-1]} --> Max Loss = {max(max_loss)}")
                print(f"-------------------------------> {colour_string} WIN = {win_count} Loss_levels = {max_loss[-1]} Max Loss = {max(max_loss)}")

                current_time = datetime.datetime.now().time()
                if current_time >= datetime.time(23, 30) and current_time >= datetime.time(1, 30):
                    print(f"\n     *** WRONG TRADING TIME ***\n     {win_count} Wins Achieved\nWe Continue Tomorrow ...")
                    break
            else:
                loss_count += 1
        # --------------------------------------------------------------------------------------------------------------
   

        # Click on the "History" tab
        driver.find_element(By.XPATH, "//a[normalize-space()='History']").click()
        time.sleep(2)
        
        # Get the history text from the webpage
        history = driver.find_element(By.XPATH, "//div[@class='p-stats__content']").text
        
        # This gets the number of times each colour appears from RED, GREEN and BLUE respectively
        # E.g '213' means 2 reds, 1 green and 3 blues
        colours_count = history.split(" ")[9:][0].split("\n")[-1]

        # This gets the Total number count for all six balls
        # E.g 148 means total sum for the given balls - 22, 14, 4, 35, 26, 47 
        sum_colours = int(history.split(" ")[9:][1].split("\n")[0])

        # Get the numbers for all 6 balls
        ball_path = "/html[1]/body[1]/div[1]/div[1]/div[1]/div[1]/main[1]/div[4]/div[2]/table[1]/tbody[1]/tr[1]/td[2]/div[1]/span["
        ball_1 = int(driver.find_element(By.XPATH, f"{ball_path}1]").text)
        ball_2 = int(driver.find_element(By.XPATH, f"{ball_path}2]").text)
        ball_3 = int(driver.find_element(By.XPATH, f"{ball_path}3]").text)
        ball_4 = int(driver.find_element(By.XPATH, f"{ball_path}4]").text)
        ball_5 = int(driver.find_element(By.XPATH, f"{ball_path}5]").text)
        ball_6 = int(driver.find_element(By.XPATH, f"{ball_path}6]").text)
        
        # Record all collected results in a listed tuple
        all_balls = (ball_1, ball_2, ball_3, ball_4, ball_5, ball_6)
        all_record = all_balls + total_stats + last_drawn_record + (colours_count, sum_colours)
        record_list.append(all_record)
        # print(all_record)

        # To open trading page
        driver.find_element(By.XPATH, "//div[@class='stats__toggle active']").click()
        time.sleep(1)

        if loss_count >= martingale_levels:
            print(f"\n      *** SORRY ***\n---> {loss_count} Losses Reached")
            # email_notification("TRADING STOPPED \nMAXIMUM LOSS REACHED ...")
            final_balance = int(driver.find_element(By.XPATH, "//div[@class='rs-menu__balance-value']").text)
            print(f"Initial Balance = {Initial_balance}")
            print(f"Final Balance = {final_balance}")
            print(f"Lost Amount = {final_balance - Initial_balance}")
            break

        if start_trade == False and (win_count >= win_count_target):
            print(f"\n    *** !!! CONGRATULATIONS !!! ***\n        {win_count} Wins Target Achieved")
            final_balance = int(driver.find_element(By.XPATH, "//div[@class='rs-menu__balance-value']").text)
            print(f"Initial Balance = {Initial_balance}")
            print(f"Fianl Balance = {final_balance}")
            print(f"Profit = {final_balance - Initial_balance}")
            break

        # START TRADING CONDITION -----------------------------------------------------------------------------
        current_time = datetime.datetime.now().time()  # breaks the program if wrong trading time -------------------------------.                        |
        if (current_time >= datetime.time(23, 45) or current_time < datetime.time(1, 30)) and start_trade == False:#             |                        |
            print(f"\n     *** WRONG TRADING TIME ***\n     {win_count} Wins Achieved - We Continue Tomorrow ...")#              |                        |
            break  # ------------------------------------------------------------------------------------------------------------' 

        colours = ("green", "red", "blue")
        temp = total_stats[1:]
        if (draw_counter >= max_num) and (temp.count(max(temp)) == 1) and ((max(temp) - (sorted(list(temp))[-2])) <= 3) and (start_trade == False):
            colour_index = temp.index(max(temp))
            colour_num = temp[colour_index]
            colour_string = colours[colour_index]
            if (max(colour_list[colour_index][-max_num:-min_num]) < colour_num) and (total_stats[0] < (colour_num + 10)) and total_stats[0] > 18:
                print(total_stats[0], "--", temp, "--->", last_drawn_record[1:], "--->> TRADE")   
                start_trade = True
                # if notification_start == False:  # For sending email notification to start trading --------------------------
                #     # email_notification("AUTO TRADING STARTED") 
                #     notification_start = True  # -------------------------------------------------------------------------
                
            elif max(colour_list[colour_index - 1][-max_num:-min_num]) > colour_num:
                print(total_stats[0], "--", temp, "-->", last_drawn_record[1:])
            else:
                print(total_stats[0], "--", temp, "-->", last_drawn_record[1:])
        else:
            print(total_stats[0], "--", temp, "-->", last_drawn_record[1:])
        
        # --------------------------------------------------------------------------------------------------------


        # START TRADING if trade condition is True -----------------------------------------------------------------------------------------------------------
        if start_trade:
            # print("START TRADE HERE ...")
            driver.find_element(By.XPATH, "//a[5]").click()  # Total Colour
            time.sleep(1)
            
            if loss_count < martingale_levels:
                stake_amount = martingale_stakes[loss_count]
                driver.find_element(By.XPATH, f"//div[@class='g-total__btn {colour_string} ']").click()  # Click on selected colour
                time.sleep(0.5)

                stake_box = driver.find_element(By.XPATH, "//body[1]/div[1]/div[1]/div[1]/div[1]/main[1]/div[2]/div[2]/div[3]/div[1]/div[2]/div[1]/div[1]/input[1]")
                stake_box.click()  # Click on stake box
                stake_box.send_keys(Keys.CONTROL + "a")  # Highlight the existing value in the stake box
                stake_box.send_keys(Keys.BACKSPACE)  # Clear the highlighted value
                time.sleep(0.5)
                
                stake_box.send_keys(stake_amount)  # Enter stake amount
                time.sleep(1)
                try:
                    driver.find_element(By.XPATH, "//a[@class='place-bet']").click()  # Place bet
                
                # if there was a problem with PLACE BET button -------------------------------------------------------------------------------------------
                except ElementClickInterceptedException:
                    print("Place bet button delay ---------------------------------->>>")
                    driver.refresh()  # Reload the current webpage
                    time.sleep(1)
                    driver.find_element(By.XPATH, "//a[5]").click()  # Total Colour
                    driver.find_element(By.XPATH, f"//div[@class='g-total__btn {colour_string} ']").click()
                    time.sleep(0.5)

                    stake_box = driver.find_element(By.XPATH, "//body[1]/div[1]/div[1]/div[1]/div[1]/main[1]/div[2]/div[2]/div[3]/div[1]/div[2]/div[1]/div[1]/input[1]")
                    stake_box.click()  # Click on stake box
                    stake_box.send_keys(Keys.CONTROL + "a")  # Highlight the existing value in the stake box
                    stake_box.send_keys(Keys.BACKSPACE)  # Clear the highlighted value
                    # time.sleep(0.5)
                    
                    stake_box.send_keys(stake_amount)  # Enter stake amount
                    time.sleep(0.5)
                    driver.find_element(By.XPATH, "//a[@class='place-bet']").click()  # Place bet
                time.sleep(3)
                driver.find_element(By.XPATH, f"//div[@class='g-total__btn {colour_string} active']").click()  # to unselect the last picked colour
                # -----------------------------------------------------------------------------------------------------------------------------------------

            else:
                print(f"Martingale Max Level reached -----> {loss_count} -- {colour_string}")

            time.sleep(3)
        # ----------------------------------------------------------------------------------------------------------------------------------------------------------------

    else:
        time.sleep(1)  # 1 seconds counter delay

print(f"\nTotal Draws = {draw_counter}")
total_minutes = (draw_counter * 50) // 60
print(f"Total Wins = {win_count}")
print(f"Total Loss = {loss_count}")
print(f"None High check count = {none_high_check}")
print(f"Time Taken = {total_minutes // 60}hrs {total_minutes % 60}mins -----------------------------------------------")

# Close the browser session
driver.quit()

Capital = 100000
Profit Target = 200
Martingale Levels - [72, 98, 133, 180, 244, 332, 450, 611, 829, 1125, 1527, 2072, 2812, 3817, 5180, 7030, 9540, 12948, 17572, 23848, 32365, 43924]
Martingale Sum = 166709
--------------------------------------------------
Initial Balance = 100000
34 -- (17, 25, 24) --> (3, 0, 5)
34 -- (16, 26, 24) --> (4, 0, 6)
34 -- (15, 27, 24) --> (5, 0, 7)
34 -- (15, 27, 24) --> (6, 1, 0)
34 -- (16, 26, 24) --> (0, 2, 1)
34 -- (17, 25, 24) --> (0, 3, 2)
35 -- (17, 24, 24) --> (1, 4, 3)
35 -- (16, 25, 24) --> (2, 0, 4)
35 -- (16, 24, 25) --> (3, 1, 0)
35 -- (16, 25, 24) --> (4, 0, 1)
35 -- (16, 25, 24) --> (5, 1, 2)
34 -- (16, 25, 25) --> (6, 2, 0)
35 -- (16, 25, 24) --> (7, 3, 1)
36 -- (16, 25, 23) --> (8, 4, 2)
36 -- (16, 25, 23) --> (9, 5, 3)
36 -- (16, 25, 23) --> (10, 6, 4)
36 -- (17, 24, 23) --> (0, 7, 5)
36 -- (17, 24, 23) --> (1, 0, 6)
36 -- (16, 24, 24) --> (2, 1, 0)
35 -- (16, 24, 25) --> (3, 2, 0)
35 -- (17, 23, 25) --> (0, 3, 1)
34 -- (17, 23, 26) ---

In [13]:
# cols=['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 
#          'green_0', 'green_2p', 'green_3p', 'green_4p', 'green_5p', 'green_6', 
#          'red_0', 'red_2p', 'red_3p', 'red_4p', 'red_5p', 'red_6', 
#          'blue_0', 'blue_2p', 'blue_3p', 'blue_4p', 'blue_5p', 'blue_6', 
#          'colours', 'total']

cols=['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 
      'none_T100', 'green_T100', 'red_T100', 'blue_T100', 
      'none_LD', 'green_LD', 'red_LD', 'blue_LD', 'colours', 'total']
df = pd.DataFrame(record_list, columns=cols)
df['colours'] = df['colours'].astype(str)

# Save the DataFrame as a CSV file
df.to_csv('colour_records/rec_34_new.csv', index=False)